In [8]:
import pandas as pd

# ! Convert Raw NOAA Data to Summarized Data

# Function to convert damage strings to numbers
def convert_damage(dmg):
    if pd.isnull(dmg) or dmg == '0':  # Check for missing values or zero
        return 0
    try:
        # Remove commas for thousands and convert to float
        dmg = dmg.replace(',', '')
        if dmg.endswith('K'):
            return float(dmg.strip('K')) * 1e3
        elif dmg.endswith('M'):
            return float(dmg.strip('M')) * 1e6
        elif dmg.endswith('B'):
            return float(dmg.strip('B')) * 1e9
        else:
            return float(dmg)
    except (ValueError, AttributeError):
        return 0

# Assuming you have these CSV files in the same directory as your Jupyter notebook, or adjust the path accordingly.
csv_files = ['weather_events_2012.csv', 'weather_events_2013.csv', 'weather_events_2014.csv', 'weather_events_2015.csv']

# Merge all CSV files into one DataFrame
df_list = [pd.read_csv("./data/noaa/" + file) for file in csv_files]
weather_data = pd.concat(df_list)

# Select relevant columns
weather_data = weather_data[['STATE', 'YEAR', 'EVENT_TYPE', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS']]

# Convert 'DAMAGE_PROPERTY' and 'DAMAGE_CROPS' to numerical values
weather_data['DAMAGE_PROPERTY'] = weather_data['DAMAGE_PROPERTY'].astype(str).apply(convert_damage)
weather_data['DAMAGE_CROPS'] = weather_data['DAMAGE_CROPS'].astype(str).apply(convert_damage)

# Group by 'STATE' and 'YEAR' and summarize the data
summary = weather_data.groupby(['STATE', 'YEAR']).agg({
    'EVENT_TYPE': 'count',
    'DAMAGE_PROPERTY': 'sum',
    'DAMAGE_CROPS': 'sum'
}).reset_index()

# Rename the columns for clarity
summary = summary.rename(columns={'EVENT_TYPE': 'NUMBER_OF_EVENTS'})

# Output the DataFrame to a new CSV file
summary.to_csv('./data/sanitized/summarized_weather_data.csv', index=False)


In [14]:
import pandas as pd

# Load the CSV file
file_path = './data/Economic Data/state_sales.csv'  

# Read the CSV file
df = pd.read_csv(file_path)

# Filter for the years 2012 to 2015
df = df[df['Year'].isin([2012, 2013, 2014, 2015])]

# Function to convert string monetary values to floats
def convert_currency(val):
    if isinstance(val, str):
        return float(val.replace(',', '').replace('"', ''))
    return val

# Convert monetary columns
columns_to_convert = ['FAH', 'FAFH', 'Total nominal food sales', 'Total constant dollar food sales']
for col in columns_to_convert:
    df[col] = df[col].apply(convert_currency)

# Output the filtered DataFrame to a new CSV file
output_file_path = './data/sanitized/processed_food_sales_data.csv'
df.to_csv(output_file_path, index=False)


    Year    State       FAH     FAFH  Total nominal food sales     FAH.1  \
15  2012  Alabama   9832.60  7500.22                  17332.82  4,946.55   
16  2013  Alabama  10204.54  7899.00                  18103.54  5,087.67   
17  2014  Alabama  10592.28  8547.42                  19139.70  5,157.77   
18  2015  Alabama  10961.59  9236.09                  20197.68  5,276.04   
39  2012   Alaska   1907.79  1364.58                   3272.37    959.76   

      FAFH.1  Total constant dollar food sales  
15  3,838.57                           8785.12  
16  3,958.14                           9045.82  
17  4,181.35                           9339.12  
18  4,392.62                           9668.67  
39    698.39                           1658.15  


In [21]:
# Python script to expand the CSV file as described

import pandas as pd

file_path = './data/Raw Data/Paper Data/regions_description.csv'

# Read the CSV file
df = pd.read_csv(file_path)

# Mapping of State FIPS codes to State Names
state_abbr_to_name = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California",
    "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia",
    "HI": "Hawaii", "ID": "Idaho", "IL": "Illinois", "IN": "Indiana", "IA": "Iowa",
    "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi", "MO": "Missouri",
    "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio",
    "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont",
    "VA": "Virginia", "WA": "Washington", "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming"
}


# Function to split multiple cities and replicate rows
def split_cities(row):
    # Split the 'Name' field into separate cities
    # Assumes the state abbreviation is the last part of the 'Name' field
    parts = row['Name'].split(',')
    state_abbr = parts[-1].strip().split('-')[-1].strip()
    state_name = state_abbr_to_name.get(state_abbr, 'Unknown')
    cities = parts[0].split('-')

    new_rows = []
    for city in cities:
        # Create a new row for each city
        new_row = row.copy()
        new_row['Name'] = city.strip()
        new_row['State'] = state_name
        new_rows.append(new_row)
    
    return new_rows

# Apply the function to each row
expanded_rows = [split_cities(row) for _, row in df.iterrows()]
flattened_rows = [item for sublist in expanded_rows for item in sublist]  # Flatten the list of lists
new_df = pd.DataFrame(flattened_rows)

# Write the processed data to a new CSV file
output_file_path = 'processed_city_data.csv'
new_df.to_csv(output_file_path, index=False)

# Optional: Display the first few rows of the new DataFrame
print(new_df.head())

    ID  State_FIPS          Name Region_type    State
0  101           1      Anniston         MSA  Alabama
0  101           1        Oxford         MSA  Alabama
0  101           1  Jacksonville         MSA  Alabama
1  102           1    Birmingham         CSA  Alabama
1  102           1        Hoover         CSA  Alabama


In [22]:
import pandas as pd

# Load the CSV files
file_path_cities = './data/sanitized/processed_city_data.csv'  
file_path_diversity = './data/Raw Data/Paper Data/diversity_US-cities.csv'  

# Read the CSV files
df_cities = pd.read_csv(file_path_cities)
df_diversity = pd.read_csv(file_path_diversity)

# Group and aggregate diversity data by ID
diversity_agg = df_diversity.groupby('ID').agg({
    'diversity_2012': 'mean',
    'diversity_2013': 'mean',
    'diversity_2014': 'mean',
    'diversity_2015': 'mean'
}).reset_index()

# Merge the city data with the aggregated diversity data
combined_df = pd.merge(df_cities, diversity_agg, on='ID', how='left')

# Output the combined DataFrame to a new CSV file
output_file_path = 'combined_city_diversity_data.csv'
combined_df.to_csv(output_file_path, index=False)

# Display the first few rows of the combined DataFrame
print(combined_df.head())


    ID  State_FIPS          Name Region_type    State  diversity_2012  \
0  101           1      Anniston         MSA  Alabama        0.633608   
1  101           1        Oxford         MSA  Alabama        0.633608   
2  101           1  Jacksonville         MSA  Alabama        0.633608   
3  102           1    Birmingham         CSA  Alabama        0.556680   
4  102           1        Hoover         CSA  Alabama        0.556680   

   diversity_2013  diversity_2014  diversity_2015  
0        0.630615        0.645494        0.638624  
1        0.630615        0.645494        0.638624  
2        0.630615        0.645494        0.638624  
3        0.541214        0.551331        0.549210  
4        0.541214        0.551331        0.549210  


In [23]:
import pandas as pd

# Load the CSV files
file_path_city_diversity = './data/Final Data/middle.csv'  
file_path_tonnage = './data/Raw Data/Paper Data/food_flow_network_US-cities.csv' 

# Read the CSV files
df_city_diversity = pd.read_csv(file_path_city_diversity)
df_tonnage = pd.read_csv(file_path_tonnage)

# Group and aggregate tonnage data by origin
tonnage_agg = df_tonnage.groupby('origin').agg({
    'tons_2012': 'sum',
    'tons_2013': 'sum',
    'tons_2014': 'sum',
    'tons_2015': 'sum'
}).reset_index()

# Merge the city diversity data with the aggregated tonnage data
combined_df = pd.merge(df_city_diversity, tonnage_agg, left_on='ID', right_on='origin', how='left')

# Drop the 'origin' column as it's redundant
combined_df = combined_df.drop('origin', axis=1)

# Output the combined DataFrame to a new CSV file
output_file_path = 'combined_city_diversity_tonnage_data.csv'
combined_df.to_csv(output_file_path, index=False)

# Display the first few rows of the combined DataFrame
print(combined_df.head())


    ID  State_FIPS          Name Region_type    State  diversity_2012  \
0  101           1      Anniston         MSA  Alabama        0.633608   
1  101           1        Oxford         MSA  Alabama        0.633608   
2  101           1  Jacksonville         MSA  Alabama        0.633608   
3  102           1    Birmingham         CSA  Alabama        0.556680   
4  102           1        Hoover         CSA  Alabama        0.556680   

   diversity_2013  diversity_2014  diversity_2015   tons_2012   tons_2013  \
0        0.630615        0.645494        0.638624  456.047652  469.026078   
1        0.630615        0.645494        0.638624  456.047652  469.026078   
2        0.630615        0.645494        0.638624  456.047652  469.026078   
3        0.541214        0.551331        0.549210  882.577600  888.952900   
4        0.541214        0.551331        0.549210  882.577600  888.952900   

    tons_2014   tons_2015  
0  472.757915  479.992386  
1  472.757915  479.992386  
2  472.757915 

In [27]:
import pandas as pd

# Load the CSV files
city_data_path = './data/Final Data/all_data_by_city.csv'  # Replace with your city data file's path
economic_data_path = './data/sanitized/processed_food_sales_data.csv'  # Replace with your economic data file's path
weather_data_path = './data/sanitized/summarized_weather_data.csv'  # Replace with your weather data file's path

# Read the CSV files
city_df = pd.read_csv(city_data_path)
economic_df = pd.read_csv(economic_data_path)
weather_df = pd.read_csv(weather_data_path)

# Duplicate city data for each year
years = [2012, 2013, 2014, 2015]
expanded_city_df = pd.concat([city_df.assign(YEAR=year) for year in years], ignore_index=True)

# Convert state names in city_df to uppercase to match other dataframes
expanded_city_df['State'] = expanded_city_df['State'].str.upper()

# Merge expanded city data with economic data
merged_df = pd.merge(expanded_city_df, economic_df, left_on=['State', 'YEAR'], right_on=['State', 'Year'], how='left')

# Merge the resulting DataFrame with weather data
final_df = pd.merge(merged_df, weather_df, left_on=['State', 'YEAR'], right_on=['STATE', 'YEAR'], how='left')

# Drop redundant columns if any
final_df = final_df.drop(['STATE', 'Year'], axis=1)

# Output the combined DataFrame to a new CSV file
output_file_path = 'final_combined_data.csv'
final_df.to_csv(output_file_path, index=False)

# Display the first few rows of the final DataFrame
print(final_df.head())


    ID  State_FIPS          Name Region_type    State  diversity_2012  \
0  101           1      Anniston         MSA  ALABAMA        0.633608   
1  101           1        Oxford         MSA  ALABAMA        0.633608   
2  101           1  Jacksonville         MSA  ALABAMA        0.633608   
3  102           1    Birmingham         CSA  ALABAMA        0.556680   
4  102           1        Hoover         CSA  ALABAMA        0.556680   

   diversity_2013  diversity_2014  diversity_2015   tons_2012  ...  YEAR  FAH  \
0        0.630615        0.645494        0.638624  456.047652  ...  2012  NaN   
1        0.630615        0.645494        0.638624  456.047652  ...  2012  NaN   
2        0.630615        0.645494        0.638624  456.047652  ...  2012  NaN   
3        0.541214        0.551331        0.549210  882.577600  ...  2012  NaN   
4        0.541214        0.551331        0.549210  882.577600  ...  2012  NaN   

   FAFH  Total nominal food sales  FAH.1  FAFH.1  \
0   NaN               

In [28]:
import pandas as pd

# Load the CSV file
file_path = './final_combined_data.csv'  # Replace with your file's path

# Read the CSV file
df = pd.read_csv(file_path)

# Function to select the correct value based on the year
def select_yearly_data(row, column_prefix):
    year = str(int(row['YEAR']))
    column_name = f"{column_prefix}_{year}"
    return row[column_name]

# Select the correct diversity and tons data for each year
df['diversity'] = df.apply(select_yearly_data, axis=1, column_prefix='diversity')
df['tons'] = df.apply(select_yearly_data, axis=1, column_prefix='tons')

# Drop the old diversity and tons columns
columns_to_drop = [col for col in df.columns if 'diversity_' in col or 'tons_' in col]
df = df.drop(columns=columns_to_drop)

# Optional: Output the adjusted DataFrame to a new CSV file
output_file_path = 'adjusted_data.csv'
df.to_csv(output_file_path, index=False)

# Display the first few rows of the adjusted DataFrame
print(df.head())


    ID  State_FIPS          Name Region_type    State  YEAR  FAH  FAFH  \
0  101           1      Anniston         MSA  ALABAMA  2012  NaN   NaN   
1  101           1        Oxford         MSA  ALABAMA  2012  NaN   NaN   
2  101           1  Jacksonville         MSA  ALABAMA  2012  NaN   NaN   
3  102           1    Birmingham         CSA  ALABAMA  2012  NaN   NaN   
4  102           1        Hoover         CSA  ALABAMA  2012  NaN   NaN   

   Total nominal food sales  FAH.1  FAFH.1  Total constant dollar food sales  \
0                       NaN    NaN     NaN                               NaN   
1                       NaN    NaN     NaN                               NaN   
2                       NaN    NaN     NaN                               NaN   
3                       NaN    NaN     NaN                               NaN   
4                       NaN    NaN     NaN                               NaN   

   NUMBER_OF_EVENTS  DAMAGE_PROPERTY  DAMAGE_CROPS  diversity        tons 

In [31]:
import pandas as pd

# Load the main CSV file and the economic data CSV file
main_data_path = './adjusted_data.csv'
economic_data_path = './data/sanitized/processed_food_sales_data.csv' 

# Read the CSV files
main_df = pd.read_csv(main_data_path)
economic_df = pd.read_csv(economic_data_path)

# Convert state names to uppercase in both DataFrames for case-insensitive matching
main_df['State'] = main_df['State'].str.upper()
economic_df['State'] = economic_df['State'].str.upper()

# Merge the main data with economic data
merged_df = pd.merge(main_df, economic_df, left_on=['State', 'YEAR'], right_on=['State', 'Year'], how='left', suffixes=('', '_y'))

# Fill the empty economic data columns in the main DataFrame
for column in ['FAH', 'FAFH', 'Total nominal food sales', 'FAH.1', 'FAFH.1', 'Total constant dollar food sales']:
    main_df[column] = main_df[column].fillna(merged_df[column + '_y'])

# Drop the duplicate columns from the merge
columns_to_drop = [col for col in main_df if col.endswith('_y')]
main_df = main_df.drop(columns=columns_to_drop)

# Optional: Output the adjusted DataFrame to a new CSV file
output_file_path = 'final_data.csv'
main_df.to_csv(output_file_path, index=False)

# Display the first few rows of the adjusted DataFrame
print(main_df.head())

    ID  State_FIPS          Name Region_type    State  YEAR     FAH     FAFH  \
0  101           1      Anniston         MSA  ALABAMA  2012  9832.6  7500.22   
1  101           1        Oxford         MSA  ALABAMA  2012  9832.6  7500.22   
2  101           1  Jacksonville         MSA  ALABAMA  2012  9832.6  7500.22   
3  102           1    Birmingham         CSA  ALABAMA  2012  9832.6  7500.22   
4  102           1        Hoover         CSA  ALABAMA  2012  9832.6  7500.22   

   Total nominal food sales     FAH.1    FAFH.1  \
0                  17332.82  4,946.55  3,838.57   
1                  17332.82  4,946.55  3,838.57   
2                  17332.82  4,946.55  3,838.57   
3                  17332.82  4,946.55  3,838.57   
4                  17332.82  4,946.55  3,838.57   

   Total constant dollar food sales  NUMBER_OF_EVENTS  DAMAGE_PROPERTY  \
0                           8785.12              1723        7445050.0   
1                           8785.12              1723        744